In [ ]:
import os
import subprocess
import pandas as pd

# Step 1: Define your features and flags
features = {
    "Soma_Surface": "-l1,2,8,1.0 -f0,0,0,10.0",
    "N_stems": "-l1,2,8,3.0 -f1,0,0,10.0",
    "N_bifs": "-l1,2,8,3.0 -f2,0,0,10.0",
    "N_branch": "-l1,2,8,3.0 -f3,0,0,10.0",
    "N_tips": "-l1,2,8,3.0 -f4,0,0,10.0",
    "Width": "-l1,2,8,3.0 -f5,0,0,10.0",
    "Height": "-l1,2,8,3.0 -f6,0,0,10.0",
    "Depth": "-l1,2,8,3.0 -f7,0,0,10.0",
    "Diameter": "-l1,2,8,3.0 -f9,0,0,10.0",
    "Length": "-l1,2,8,3.0 -f11,0,0,10.0",
    "Surface": "-l1,2,8,3.0 -f12,0,0,10.0",
    "Volume": "-l1,2,8,3.0 -f14,0,0,10.0",
    "EucDistance": "-l1,2,8,3.0 -f15,0,0,10.0",
    "PathDistance": "-l1,2,8,3.0 -f16,0,0,10.0",
    "Branch_Order": "-l1,2,8,3.0 -f18,0,0,10.0",
    "Terminal_degree": "-l1,2,8,3.0 -f19,0,0,10.0",
    "Branch_pathlength": "-l1,2,8,3.0 -f23,0,0,10.0",
    "Contraction": "-l1,2,8,3.0 -f24,0,0,10.0",
    "Fragmentation": "-l1,2,8,3.0 -f25,0,0,10.0",
    "Partition_asymmetry": "-l1,2,8,3.0 -f28,0,0,10.0",
    "Pk_classic": "-l1,2,8,3.0 -f31,0,0,10.0",
    "Bif_ampl_local": "-l1,2,8,3.0 -f33,0,0,10.0",
    "Bif_ampl_remote": "-l1,2,8,3.0 -f34,0,0,10.0",
    "Bif_tilt_local": "-l1,2,8,3.0 -f35,0,0,10.0",
    "Bif_tilt_remote": "-l1,2,8,3.0 -f36,0,0,10.0",
    "Bif_torque_local": "-l1,2,8,3.0 -f37,0,0,10.0",
    "Bif_torque_remote": "-l1,2,8,3.0 -f38,0,0,10.0",
    "Helix": "-l1,2,8,3.0 -f43,0,0,10.0",
    "Fractal_Dim": "-l1,2,8,3.0 -f44,0,0,10.0",
    "Branch_pathlength": "-l1,2,8,3.0 -l1,2,19,1.0  -f23,0,0,10.0 ", # Branch Pathlength of Terminal Branches
    "Contraction": "-l1,2,8,3.0 -l1,2,19,1.0  -f24,0,0,10.0" # Contraction of Terminal Branches
}

# Paths
swc_dir = r"C:\Users\MasoodAkram\Desktop\MorphoMeasure_2.0\swc_files"
output_dir = r"C:\Users\MasoodAkram\Desktop\MorphoMeasure_2.0\Measurements"
tmp_dir = r"C:\Users\MasoodAkram\Desktop\MorphoMeasure_2.0\tmp"
lm_exe_path = r"C:\Users\MasoodAkram\Desktop\MorphoMeasure_2.0\Lm.exe"

# Ensure output and tmp directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)

# Step 2: Process each SWC file
for swc_filename in os.listdir(swc_dir):
    if swc_filename.endswith(".swc"):
        swc_path = os.path.join(swc_dir, swc_filename)
        swc_base = os.path.splitext(swc_filename)[0]
        print(f"\n🧠 Processing SWC file: {swc_filename}")

        # Container for this file's morphometrics
        feature_dfs = []

        # Step 3: Run each feature
        for feature_name, feature_flags in features.items():
            print(f"🔧 Feature: {feature_name}")

            temp_output_path = os.path.join(tmp_dir, f"{swc_base}_{feature_name}.csv")
            lmin_path = os.path.join(tmp_dir, "Lmin.txt")

            # Build Lmin.txt content
            param_lines = f"{feature_flags}\n-s{temp_output_path} -R\n{swc_path}\n"
            with open(lmin_path, "w") as f:
                f.write(param_lines)

            # Run LMeasure
            result = subprocess.run([lm_exe_path, lmin_path], capture_output=True, text=True)

            # Clean and read feature output
            if os.path.exists(temp_output_path):
                try:
                    # Read and clean: drop non-numeric rows (like the last metadata line)
                    df_raw = pd.read_csv(temp_output_path, header=None)
                    df_clean = df_raw[pd.to_numeric(df_raw[0], errors='coerce').notna()]
                    df_clean.columns = [feature_name]
                    feature_dfs.append(df_clean.reset_index(drop=True))
                    print(f"✅ Loaded {len(df_clean)} clean values")
                except Exception as e:
                    print(f"⚠️ Error reading {feature_name}: {e}")
            else:
                print(f"❌ No output for {feature_name}")

        # Step 4: Save final morphometrics for this SWC
        if feature_dfs:
            df_combined = pd.concat(feature_dfs, axis=1)
            morpho_outfile = os.path.join(output_dir, f"Branch_Morphometrics_{swc_base}.csv")
            df_combined.to_csv(morpho_outfile, index=False)
            print(f"📁 Saved: {morpho_outfile}")
        else:
            print(f"⚠️ No features extracted for {swc_filename}")

# Step 5: Clean up tmp folder
for fname in os.listdir(tmp_dir):
    if fname.endswith(".csv"):
        try:
            os.remove(os.path.join(tmp_dir, fname))
        except Exception as e:
            print(f"⚠️ Failed to delete {fname}: {e}")
print("\n🧹 All temporary CSVs cleaned up from tmp folder.")



🧠 Processing SWC file: 0001_1060-5814-1310.CNG.swc
🔧 Feature: Soma_Surface
✅ Loaded 2 clean values
🔧 Feature: N_stems
✅ Loaded 12 clean values
🔧 Feature: N_bifs
✅ Loaded 38 clean values
🔧 Feature: N_branch
✅ Loaded 88 clean values
🔧 Feature: N_tips
✅ Loaded 50 clean values
🔧 Feature: Width
✅ Loaded 1629 clean values
🔧 Feature: Height
✅ Loaded 1629 clean values
🔧 Feature: Depth
✅ Loaded 1629 clean values
🔧 Feature: Diameter
✅ Loaded 1629 clean values
🔧 Feature: Length
✅ Loaded 1629 clean values
🔧 Feature: Surface
✅ Loaded 1629 clean values
🔧 Feature: Volume
✅ Loaded 1629 clean values
🔧 Feature: EucDistance
✅ Loaded 1629 clean values
🔧 Feature: PathDistance
✅ Loaded 1629 clean values
🔧 Feature: Branch_Order
✅ Loaded 1629 clean values
🔧 Feature: Terminal_degree
✅ Loaded 1629 clean values
🔧 Feature: Branch_pathlength
✅ Loaded 88 clean values
🔧 Feature: Contraction
✅ Loaded 88 clean values
🔧 Feature: Fragmentation
✅ Loaded 88 clean values
🔧 Feature: Partition_asymmetry
✅ Loaded 38 clean va